<a href="https://colab.research.google.com/github/rhudaina/Linear-Systems-and-Applications-A-Hands-On-Python-Workshop/blob/main/Day2/Day2_Lecture_1_DirectSolvers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we wish to solve linear system
$$Ax = y$$

# BLAS and LAPACK

Regardless of what language you're using, chances are if you're doing numerical linear algebra, you are able to take advantage of libraries of code which implement most common linear algebra routines and factorizations.
* [BLAS (Basic Linear Algebra Subprograms)](https://www.netlib.org/blas/) - routines that provide standard building blocks for performing basic vector and matrix operations
* [LAPACK (Linear Algebra PACKage)](https://www.netlib.org/lapack/) - routines for solving systems of simultaneous linear equations, least-squares solutions of linear systems of equations, eigenvalue problems, and singular value problems


The [NumPy linear algebra](https://numpy.org/doc/stable/reference/routines.linalg.html#module-numpy.linalg) functions rely on BLAS and LAPACK to provide efficient low level implementations of standard linear algebra algorithms.

You can view what BLAS and LAPACK libraries NumPy is using

In [ ]:
import numpy as np

np.__config__.show()

[SciPy (Scientific Python)](https://docs.scipy.org/doc/scipy/tutorial/index.html) is a collection of mathematical algorithms and convenience functions built on NumPy. It adds significant power to Python by providing the user with high-level commands and classes for manipulating and visualizing data.

The SciPy library also contains a linalg submodule, and there is overlap in the functionality provided by the SciPy and NumPy submodules.

In [ ]:
import scipy as sp

# Matrix Factorizations


A **matrix factorization** or **matrix decomposition** writes a matrix $A$ as the product of matrices $A = BCD\dots$, where the matrices in the product typically have some special structure.

For example:
* Diagonal matrices - easy to apply and solve linear systems
* Triangular matrices (upper or lower) - fast to solve linear systems
* Orthonormal matrices: $Q$ orthogonal means $Q^T = Q^{\dagger}$ (pseudoinverse)
* Permutation matrices: sparse orthonormal matrices

Most of the matrix factorizations we will see run in $O(n^3)$ time for a $n\times n$ matrix $A$, or $O(\min(m,n)^2 \max(m,n))$ for a $m\times n$ matrix $A$.

## LU Factorization (for square matrices)


For numerical stability, this is often computed with a *pivoting* strategy, which means there is also row or column permutation matrix $P$ in the factorization:
$$ A = PLU$$
where $L$ is lower-triangular and $U$ is upper triangular.  

In [ ]:
n = 1000
A = np.random.randn(n, n)
x = np.random.rand(n)
y = A @ x

P, L, U = sp.linalg.lu(A)

In [ ]:
sp.linalg.norm(P @ L @ U - A)

The nice thing about triangular matrices is that they can solve linear systems in $O(n^2)$ time, instead of $O(n^3)$ time for general matrices, using the forward or backward substitution algorithms via a special function `solve_triangular`. See [documentation](https://docs.scipy.org/doc/scipy-1.13.0/reference/generated/scipy.linalg.solve_triangular.html)

In [ ]:
from scipy.linalg import solve_triangular

z = solve_triangular(L, P.T @ y, lower=True)  # forward substitution
x_lu = solve_triangular(U, z, lower=False)    # backward substitution

In [ ]:
sp.linalg.norm(x_lu - x)

Let us define a function that solves a given linear system using LU factorization and check its computational cost.

In [ ]:
def solve_lu(A, y):
  ''' solves Ax = y using LU factorization '''
  P, L, U = sp.linalg.lu(A)
  z = solve_triangular(L, P.T @ y, lower=True)
  x = solve_triangular(U, z, lower=False)
  return x

In [ ]:
%timeit solve_lu(A,y)

Alternatively, we can do the following:

In [ ]:
k = 7
size = [2**(i+4) for i in range(k)]
print(size)

In [ ]:
import time

time_lu = np.zeros(k)

for i in range(k):
  n = 2**(i+4)
  A = np.random.randn(n, n)
  x = np.random.rand(n)
  y = A @ x

  t_start = time.time()
  x_lu = solve_lu(A,y)
  t_end = time.time()
  time_lu[i] = t_end - t_start

In [ ]:
import matplotlib.pyplot as plt

plt.plot(size, time_lu)
plt.xscale('log', base=2)
plt.xlabel('size of matrix')
plt.ylabel('time')
plt.show()

## QR Factorization

We decompose matrix $$A = QR$$ into a matrix $Q$ with orthonormal columns, and an upper triangular matrix $R$.  

In [ ]:
n = 1000
m = 500
A = np.random.randn(n, m)

Q, R = np.linalg.qr(A)
print(Q.shape, R.shape)

np.linalg.norm(Q @ R  - A)

In [ ]:
Q, R = np.linalg.qr(A, mode='complete')
print(Q.shape, R.shape)

np.linalg.norm(Q @ R  - A)

Using `scipy`:

In [ ]:
Q, R = sp.linalg.qr(A, mode='economic')
print(Q.shape, R.shape)

sp.linalg.norm(Q @ R  - A)

The QR factorization is used for least-squares solutions.

In [ ]:
def solve_qr(A, y):
  ''' solves least squares problem ||Ax - y|| using QR factorization '''
  Q, R = sp.linalg.qr(A, mode='economic')
  z = Q.T @ y
  x = solve_triangular(R, z, lower=False)
  return x

In [ ]:
x = np.random.rand(m)
y = A @ x

x_qr = solve_qr(A, y)
sp.linalg.norm(x_qr - x)

In [ ]:
k = 7
size = [2**(i+4) for i in range(k)]
print(size)

In [ ]:
import time

time_qr = np.zeros(k)

for i in range(k):
  n = 2**(i+4)
  m = 2**(i+3)
  A = np.random.randn(n, m)
  x = np.random.rand(m)
  y = A @ x

  t_start = time.time()
  x_qr = solve_qr(A,y)
  t_end = time.time()
  time_qr[i] = t_end - t_start

In [ ]:
import matplotlib.pyplot as plt

plt.plot(size, time_qr)
plt.xscale('log', base=2)
plt.xlabel('size of matrix')
plt.ylabel('time')
plt.show()

## Eigenvalue Decomposition

A vector $x$ is an eigenvector of $A$ with eigenvalue $\lambda$ if $Ax = x \lambda$.  An eigenvalue decomposition is a decomposition $A = X \Lambda X^{-1}$ where $\Lambda$ is a diagonal matrix.  We can compute such a decomposition using `eig`:

In [ ]:
n = 1000
A = np.random.randn(n, n)
Lam, X = sp.linalg.eig(A)

The columns of `X` are eigenvectors, and eigenvalues are diagonal entries of `Lam`

In [ ]:
x = X[:,0]
sp.linalg.norm(A @ x - Lam[0] * x)

When `A` is symmetric (or Hermitian), there exists and orthonormal basis where every basis element is an eigenvector.  In this case, we can write $A = U\Lambda U^H$.  There is a special function `eigh` for such a situation.

In [ ]:
A = np.random.randn(n,n)
A = A + A.T               # make symmetric
Lam, U = sp.linalg.eigh(A)

In [ ]:
x = U[:,0]
sp.linalg.norm(A @ x - Lam[0] * x)

In [ ]:
sp.linalg.norm(U @ U.T - np.eye(n))

Both eigenvector decompositions `eig` and `eigh` take $O(n^3)$ time for a $n\times n$ matrix.

Let us investigate which is faster in practice on a symmetric matrix.

In [ ]:
k = 7
size = [2**(i+4) for i in range(k)]
print(size)

In [ ]:
import time

t_eig = np.zeros(k)
t_eigh = np.zeros(k)


for i in range(k):
  n = 2**(i+4)
  A = np.random.randn(n,n)
  A = A + A.T

  t_start = time.time()
  _, X = sp.linalg.eig(A)
  t_end = time.time()
  t_eig[i] = t_end - t_start

  t_start = time.time()
  _, U = sp.linalg.eigh(A)
  t_end = time.time()
  t_eigh[i] = t_end - t_start

In [ ]:
import matplotlib.pyplot as plt

plt.plot(size, t_eig, label ='eig')
plt.plot(size, t_eigh, label ='eigh')
plt.xscale('log', base=2)
plt.xlabel('size of matrix')
plt.ylabel('time')
plt.legend()
plt.show()

Solving $Ax = y$ translates to solving
$$X \Lambda X^{-1}x = y.$$
Since $X$ is orthogonal, then $XX^T = X^TX = I$. Hence, we have
$$x = X\Lambda^{-1}X^Ty$$
where $\Lambda^{-1}$ is a diagonal matrix whose diagonal entries are the reciprocal of the eigenvalues.


In [ ]:
def solve_eigh(A, y):
  ''' solves Ax = y using eigenvalue decomposition with eigh '''
  Lam, U = sp.linalg.eigh(A)
  x = U @ np.diag(1/Lam) @ U.T @ y
  return x

In [ ]:
import time

time_lu = np.zeros(k)
time_qr = np.zeros(k)
time_eigh = np.zeros(k)

for i in range(k):
  n = 2**(i+4)
  A = np.random.randn(n,n)
  A = A + A.T
  x = np.random.randn(n)
  y = A @ x

  t_start = time.time()
  x = solve_lu(A,y)
  t_end = time.time()
  time_lu[i] = t_end - t_start

  t_start = time.time()
  x = solve_qr(A,y)
  t_end = time.time()
  time_qr[i] = t_end - t_start

  t_start = time.time()
  x = solve_eigh(A,y)
  t_end = time.time()
  time_eigh[i] = t_end - t_start

In [ ]:
import matplotlib.pyplot as plt

plt.plot(size, time_lu, label ='LU')
plt.plot(size, time_qr, label ='QR')
plt.plot(size, time_eigh, label ='eigh')
plt.xscale('log', base=2)
plt.yscale('log', base=10)
plt.xlabel('size of matrix')
plt.ylabel('time')
plt.legend()
plt.show()

## SVD

The singular value decomposition is an extremely useful practical and theoretical tool.  We can decompose a $m\times n$ matrix $A$ as $A = U \Sigma V^T$, where $U$ is a $m \times m$ matrix with orthonormal columns (called left singular vectors), $V$ is a $n\times n$ matrix with orthonormal columns (called right singular vectors), and $\Sigma$ is a diagonal matrix with positive entries decreasing in magnitude (called singular values).

The top singular value solves the variational problem $\sigma_0 = \max u^T A v$ subject to $\|u\|_2 = 1, \|v\|_2=1$, and describes the direction in which $A$ induces the largest change in maginitude in a vector. The next singular value is defined similarly on the subspaces orthogonal to $u$ and $v$, and so on.

One way to visualize the action of a matrix is seeing how it maps the unit sphere.  The image is an ellipsoid, and the right singular vectors give the directions of the axes, and the singular values give the lengths of these axes.

In [ ]:
n = 2
A = np.random.randn(n,n)
U, S, Vt = sp.linalg.svd(A)

In [ ]:
theta = np.linspace(0, 2*np.pi, num=200, endpoint=True)
xx = np.vstack((np.cos(theta), np.sin(theta)))
yy = A @ xx

plt.plot(yy[0], yy[1])
plt.scatter(S*U[0], S*U[1])
plt.axis('equal')
plt.show()

Computing the SVD takes $O(n^3)$ time for a $n\times n$ matrix, just like all the other matrix factorizations we've seen.

In [ ]:
n = 1000
A = np.random.randn(n, n//5)

U, S, Vh = np.linalg.svd(A)
print(U.shape, S.shape, Vh.shape)

In [ ]:
U, S, Vh = np.linalg.svd(A, full_matrices=False)
print(U.shape, S.shape, Vh.shape)

Using `scipy`:

In [ ]:
U, S, Vh = sp.linalg.svd(A)
print(U.shape, S.shape, Vh.shape)

In [ ]:
U, S, Vh = sp.linalg.svd(A, full_matrices = False)
print(U.shape, S.shape, Vh.shape)

sp.linalg.norm(U @ np.diag(S) @ Vh  - A)

Define a function that solves linear system $Ax = y$ using SVD

In [ ]:
def solve_svd(A, y):
  ''' solves Ax = y using SVD '''
  U, S, Vh = sp.linalg.svd(A, full_matrices = False)
  x = Vh.T @ np.diag(1/S) @ U.T @ y
  return x

In [ ]:
x = np.random.randn(n//5)
y = A @ x
x_svd = solve_svd(A,y)
sp.linalg.norm(x_svd - x)

### Example 1

In [ ]:
n = 50;

# setup true data
m = 3;
b = 2;
t = np.linspace(-2,2,n);
ytrue = m*t + b;

# generate noisy data
A = np.ones((n,2));
A[:,0] = t;
y = ytrue + np.random.randn(n);

x = solve_svd(A, y)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(t,ytrue,'b',label='True Line');
plt.plot(t,y,'r.',label='Noisy Data');
plt.plot(t,np.dot(A,x),'k--',label='Fitted Line');
plt.legend()
plt.show()

### Example 2

In [ ]:
A = np.array([[7,26,6,60],
              [1,29,15,52],
              [11,56,8,20],
              [11,31,8,47],
              [7,52,6,33],
              [11,55,9,22],
              [3,71,17,6],
              [1,31,22,44],
              [2,54,18,22],
              [21,47,4,26],
              [1,40,23,34],
              [11,66,9,12],
              [10,68,8,12]]);
print("A =\n",A);

y = np.array([78.5, 74.3, 104.3, 87.6, 95.9, 109.2, 102.7, 72.5, 93.1, 115.9, 83.8, 113.3, 109.4]);
print("y = ",y)

In [ ]:
x = solve_svd(A,y)
print(x)

In [ ]:
x = np.linalg.pinv(A) @ y
print(x)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(y,'b',label='Heat Data');
plt.plot(np.matmul(A,x),'r',label='Solution');
plt.xlabel("Cement mixtures");
plt.ylabel("Heat");
plt.legend()
plt.show()

### Example 3. BVP via FDM

$$\begin{cases} -u'' = \frac{\pi}{2}\sin\big(\frac{\pi}{2}\big)\qquad 0\leq x\leq 1\\u(0) = 0\\ u(1) = 0 \end{cases}$$

# Reference

1.   [Brad Nelson (2021), Scientific Computing with Python](https://caam37830.github.io/book/index.html)

